In [9]:
from math import sqrt
import random
from sklearn.cluster import KMeans
import math
import collections
import matplotlib.pyplot as plt
import numpy as np

In [10]:
class People:
    def __init__(self):
        self.destination = [0, 0]
        self.destinationRoadIndex = 0
        self.route = []
        self.passWayPoint = False
        self.wayPoint = [0,0]

        startRoadIndex = random.randint(0, 3)
        if (startRoadIndex % 2) == 0:
            self.location = [
                roadLocation[startRoadIndex]["centerPos"][0] + random.randint(-9, 9),
                roadLocation[startRoadIndex]["centerPos"][1]
            ]
        else:
            self.location = [
                roadLocation[startRoadIndex]["centerPos"][0],
                roadLocation[startRoadIndex]["centerPos"][1] + random.randint(-9, 9),
            ]
        self.roadIndex = getRoadLocation(self.location)

    def getDestination(self):
        return self.destination

    def setDestination(self, x, y):
        self.destination = [x , y]
        self.destinationRoadIndex = getRoadLocation(self.destination)
        self.wayPoint = getProximatePos(self.location, roadLocation[self.destinationRoadIndex]['location'])
    
    def getDestinationRoadIndex(self):
        return self.destinationRoadIndex
    
    def getRoute(self):
        return self.route

    def setRoute(self, route):
        self.route = route

    def getLocation(self):
        return self.location
    
    def setLocation(self, x, y):
        self.location = [x, y]
        
    def getPassWayPointBool(self):
        return self.passWayPoint
    
    def setPassWayPointBool(self, boolVar):
        self.passWayPoint = boolVar
    
    def getRoadIndex(self):
        return self.roadIndex
    
    def setRoadIndex(self, index):
        self.roadIndex = index
    
    def getWayPoint(self):
        return self.wayPoint
    
    def setWsyPoint(self, pos):
        self.wayPoint = pos

In [11]:
def getDistance(x,y):
    a = x[0] - y[0]
    b = x[1] - y[1]
    
    return sqrt(a**2 + b**2)

def getProximatePos(peopleLoc, locList):
    tempVar = 0
    resultIndex = 0
    
    for tempLoc in locList:
        distance = getDistance(peopleLoc, tempLoc)
        if tempVar != 0 and distance < tempVar:
            tempVar = distance
            resultIndex = locList.index(tempLoc)
    
    return locList[resultIndex]

In [12]:
def getRoute_a(a_pos,b_pos):
    if (a_pos[0]-b_pos[0]) == 0: # x 축이 0일때
        x = a_pos[0]
        y = a_pos[1]
        if a_pos[1]-b_pos[1] > 0:
            return [x,y-1]
        else:
            return [x,y+1]
    elif (a_pos[1]-b_pos[1]) == 0: # y 축이 0일떄
        x = a_pos[0]
        y = a_pos[1]
        if a_pos[0]-b_pos[0] > 0:
            return [x-1,y]
        else:
            return [x+1,y]
    else:
        w = (a_pos[1]-b_pos[1])/(a_pos[0]-b_pos[0])
        if (a_pos[0]-b_pos[0]) < 0:
            if (a_pos[1]-b_pos[1]) < 0:
                x = sqrt(1/((w)**2+1)) + a_pos[0]
                y = sqrt(1-(1/((w)**2+1))) + a_pos[1]
            else:
                x = sqrt(1/((w)**2+1)) + a_pos[0]
                y = -sqrt(1-(1/((w)**2+1))) + a_pos[1]
        else:
            if (a_pos[1]-b_pos[1]) < 0:
                x = -sqrt(1/((w)**2+1)) + a_pos[0]
                y = sqrt(1-(1/((w)**2+1))) + a_pos[1]
            else:
                x = -sqrt(1/((w)**2+1)) + a_pos[0]
                y = -sqrt(1-(1/((w)**2+1))) + a_pos[1]
        


        return [x,y]

def getRoute(a,b):
    k=[]
    n = getRoute_a(a,b)
    k.append(n)
    a_0 = a[0]
    b_0 = b[0]
    a_1 = a[1]
    b_1 = b[1]
    k_0 = k[-1][0]
    k_1 = k[-1][1]    

    while (min(a_0,b_0) <= k_0 <= max(a_0,b_0)) and (min(a_1,b_1) <= k_1 <= max(a_1,b_1)):
        if k_0 in [min(a_0,b_0),max(a_0,b_0)] and k_1 in [min(a_1,b_1),max(a_1,b_1)]:
            break
            
        a = getRoute_a(a,b)
        k.append(a)
        k_0 = k[-1][0]
        k_1 = k[-1][1]

        
    
    del k[0]
    if len(k) > 1:
        del k[-1]
    
    k.append(b)

    return k

In [13]:
#1m이내 사람 측정 코드
def find_ev(people):
    center_value = []

    for k in [people]:
        km = KMeans(1)
        km.fit(k)
        center_value.append(km.cluster_centers_)
        
    return center_value[0][0].tolist()

def distance(k_1,k_2):
    Point2D = collections.namedtuple('Point2D', ['x', 'y'])    
    p1 = Point2D(x=k_1[0], y=k_1[1])    # 점1
    p2 = Point2D(x=k_2[0], y=k_2[1])    # 점2
    a = p1.x - p2.x    # 선 a의 길이
    b = p1.y - p2.y    # 선 b의 길이
    c = math.sqrt((a * a) + (b * b))
    return c

def keep_distance(people):
    m = []
    c = []
    for target in people:
        for cn in people:
            if distance(cn,target) <=1:
                m.append(cn)
        point = find_ev(m)
        c.append(point)
        m = []

    return c

In [14]:
def getRoadLocation(peopleNowLoc):
    for roadInfo in roadLocation:
        checkLoc = roadInfo["location"]
        
        xpos = False
        ypos = False
        
        maxLoc = 0
        minLoc = 0
        
        #X좌표
        if checkLoc[0][0] < checkLoc[2][0]:
            maxLoc = checkLoc[2][0]
            minLoc = checkLoc[0][0]
        else:
            maxLoc = checkLoc[0][0]
            minLoc = checkLoc[2][0]
        if minLoc <= peopleNowLoc[0] and maxLoc >= peopleNowLoc[0]:
            xpos = True
        
        #Y좌표
        if checkLoc[0][1] < checkLoc[2][1]:
            maxLoc = checkLoc[2][1]
            minLoc = checkLoc[0][1]
        else:
            maxLoc = checkLoc[0][1]
            minLoc = checkLoc[2][1]
        if minLoc <= peopleNowLoc[1] and maxLoc >= peopleNowLoc[1]:
            ypos = True
            
        
        if xpos and ypos:
            return roadLocation.index(roadInfo)
    

    return -1

In [15]:
peopleAmount = 100
peopleRadius = 0.3
roadDirection = [ #0은 원점으로, 1은 자기 위치 끝쪽으로 | 골목 방향 설정 Var
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
    [1,0,1,1],
]
roadLocation = [
    {
        "centerPos": [0, 50],
        "location": [
            [-10, 10],
            [-10, 50],
            [10, 50],
            [10, 10]
        ],
    },
    {
        "centerPos": [50, 0],
        "location": [
            [10, 10],
            [50, 10],
            [50, -10],
            [10, -10]
        ],
    },
    {
        "centerPos": [0, -50],
        "location": [
            [10, -10],
            [10, -50],
            [-10, -50],
            [-10, -10]
        ],
    },
    {
        "centerPos": [-50, 0],
        "location": [
            [-10, -10],
            [-50, -10],
            [-50, 10],
            [-10, 10]
        ],
    },
]

In [16]:
def setPeopleLocation(people, direction):
    roadIndex = people.getRoadIndex()
    
    if direction[roadIndex] == 0:
        if people.getPassWayPointBool():
            people.setRoute(getRoute(people.getLocation(), people.getDestination()))
        else:
            people.setRoute(getRoute(people.getLocation(), [0,0]))
    else:
        people.setRoute(getRoute(people.getLocation(), roadLocation[roadIndex]["centerPos"]))


peopleList = []
for people in range(0, peopleAmount):
    peopleModel = People()
    roadIndex = random.randint(0, 3)
    peopleModel.setDestination(roadLocation[roadIndex]["centerPos"][0], roadLocation[roadIndex]["centerPos"][1])
    peopleModel.setRoute(getRoute(peopleModel.getLocation(), peopleModel.getDestination()))

    peopleList.append(peopleModel)

print("사람 생성 완료")

for direction in roadDirection: #n번 횟수 시뮬레이션 반복
    peoplePosList = []
    for people in peopleList:
        peoplePosList.append(people.getLocation())
    preprocessingPosList = keep_distance(peoplePosList)
    
    
    roadPeopleAmount = {
        0: 0,
        1: 0,
        2: 0,
        3: 0,
        -1: 0,
    }
    for people in peopleList: #사람마다 계산
        peopleIndex = peopleList.index(people)
        if peoplePosList[peopleIndex] == preprocessingPosList[peopleIndex]: #1m이내에 사람이 없어 사람을 피하기 위해 움직이지 않았을시
            if people.getDestination() == people.getLocation():
                newRoadIndex = random.randint(0, 3)
                newPos = roadLocation[newRoadIndex]["centerPos"]
                people.setRoadIndex(newRoadIndex)
                people.setDestination(newPos[0], newPos[1])
                people.setPassWayPointBool(False)

                if people.getPassWayPointBool():
                    people.setRoute(getRoute(people.getLocation(), people.getDestination()))
                else:
                    people.setRoute(getRoute(people.getLocation(), people.getWayPoint()))
                    
            else:
                tempLoc = people.getRoute()
                if len(tempLoc) == 0:
                    people.setRoute(getRoute(people.getLocation(), people.getDestination()))
                tempLoc = people.getRoute()
                                    
                people.setRoute((people.getRoute())[1:])
                people.setLocation(tempLoc[0][0], tempLoc[0][1])
        else: #1m이내에 사람이 있어 사람을 피하기 위해 움직였을시
            tempLoc = preprocessingPosList[peopleIndex]
            people.setLocation(tempLoc[0], tempLoc[1])
            
            if people.getPassWayPointBool():
                people.setRoute(getRoute(people.getLocation(), people.getDestination()))
            else:
                people.setRoute(getRoute(people.getLocation(), people.getWayPoint()))
        
        if people.getLocation() == people.getWayPoint(): #목적지에 도달하는 과정에서 원점에 도착하였을 경우
            people.setPassWayPointBool(True)
            
        
        
        roadIndex = getRoadLocation(people.getLocation())
        roadPeopleAmount[roadIndex] = roadPeopleAmount[roadIndex]+1
        
        
        
    print(roadPeopleAmount)
        

사람 생성 완료
{0: 26, 1: 28, 2: 26, 3: 20, -1: 0}
{0: 26, 1: 28, 2: 26, 3: 20, -1: 0}
{0: 26, 1: 28, 2: 26, 3: 20, -1: 0}
{0: 26, 1: 28, 2: 26, 3: 20, -1: 0}
{0: 26, 1: 28, 2: 26, 3: 20, -1: 0}
{0: 26, 1: 28, 2: 26, 3: 20, -1: 0}
{0: 26, 1: 28, 2: 26, 3: 20, -1: 0}



KeyboardInterrupt

